In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import datetime as dt
import numpy as np
import copy

## Reading excel file


In [2]:
xls = pd.ExcelFile('data.xlsx')

## Reading individual Sheets

In [3]:
# Industrialisation DataFrame
ind = pd.read_excel(xls, 'industrialisation')
ind.head()
ind.shape

(2840, 30)

In [4]:
# CO2 dataframe
co2 = pd.read_excel(xls, 'Co2')
co2

,Entity,Code,Year,Annual CO2 emissions (tonnes )
0,Afghanistan,AFG,1949,14656.00
1,Afghanistan,AFG,1950,84272.00
2,Afghanistan,AFG,1951,91600.00
3,Afghanistan,AFG,1952,91600.00
4,Afghanistan,AFG,1953,106256.00
...,...,...,...,...
20848,Zimbabwe,ZWE,2013,11536239.29
20849,Zimbabwe,ZWE,2014,11866348.41
20850,Zimbabwe,ZWE,2015,10907603.94
20851,Zimbabwe,ZWE,2016,9932649.88


In [5]:
co2.shape

(20853, 4)

In [6]:
# Global Temps Dataframe
gltemp = pd.read_excel(xls, 'GlobalLandTemperaturesByCity')
gltemp.head()
gltemp.shape

(1048575, 7)

## Cleaning the DFs

### Cleaning Global Temps By City

In [7]:
gltemp.isna().sum()

dt                                   0
AverageTemperature               47547
AverageTemperatureUncertainty    47547
City                                 0
Country                              0
Latitude                             0
Longitude                            0
dtype: int64

In [8]:
gltemp.iloc[798583]['dt']

datetime.datetime(1900, 7, 1, 0, 0)

In [9]:
gltemp = gltemp.dropna()

In [10]:
gltemp.isna().sum()

dt                               0
AverageTemperature               0
AverageTemperatureUncertainty    0
City                             0
Country                          0
Latitude                         0
Longitude                        0
dtype: int64

In [11]:
print(gltemp.iloc[798580])

dt                               1987-07-01 00:00:00
AverageTemperature                             22.97
AverageTemperatureUncertainty                  0.207
City                                           Benxi
Country                                        China
Latitude                                      40.99N
Longitude                                    123.55E
Name: 836224, dtype: object


In [12]:
gltemp = gltemp.reset_index()

In [13]:
del gltemp['index']

In [14]:
gltemp.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Ã…rhus,Denmark,57.05N,10.33E
1,1744-04-01,5.788,3.624,Ã…rhus,Denmark,57.05N,10.33E
2,1744-05-01,10.644,1.283,Ã…rhus,Denmark,57.05N,10.33E
3,1744-06-01,14.051,1.347,Ã…rhus,Denmark,57.05N,10.33E
4,1744-07-01,16.082,1.396,Ã…rhus,Denmark,57.05N,10.33E


In [15]:
gltemp['dt'] = pd.to_datetime(gltemp['dt'])


In [16]:
gltemp.head()


,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Ã…rhus,Denmark,57.05N,10.33E
1,1744-04-01,5.788,3.624,Ã…rhus,Denmark,57.05N,10.33E
2,1744-05-01,10.644,1.283,Ã…rhus,Denmark,57.05N,10.33E
3,1744-06-01,14.051,1.347,Ã…rhus,Denmark,57.05N,10.33E
4,1744-07-01,16.082,1.396,Ã…rhus,Denmark,57.05N,10.33E


In [17]:
gltemp = gltemp[gltemp['dt'] > pd.Timestamp(year=1989, day=31, month=12)]

In [18]:
gltemp.head()
gltemp.shape

(119879, 7)

In [19]:
gltemp = gltemp.reset_index(drop=True)

In [27]:
#gltemp['dt'] = gltemp['dt'].dt.year
gltemp

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1990,4.267,0.459,Ã…rhus,Denmark,57.05N,10.33E
1,1990,5.895,0.460,Ã…rhus,Denmark,57.05N,10.33E
2,1990,6.565,0.299,Ã…rhus,Denmark,57.05N,10.33E
3,1990,8.068,0.222,Ã…rhus,Denmark,57.05N,10.33E
4,1990,12.944,0.287,Ã…rhus,Denmark,57.05N,10.33E
...,...,...,...,...,...,...,...
119874,2013,27.745,0.241,Bontang,Indonesia,0.80N,118.13E
119875,2013,27.795,0.258,Bontang,Indonesia,0.80N,118.13E
119876,2013,28.220,0.292,Bontang,Indonesia,0.80N,118.13E
119877,2013,27.127,0.231,Bontang,Indonesia,0.80N,118.13E


In [28]:
# List of countries being analysed
countries = ['India', 'China', 'United States', 'United Kingdom', 'Japan', 'Germany']

country = gltemp[gltemp['Country'].isin(countries)]

In [29]:
country = country.reset_index(drop=True)
country

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1990,-13.093,0.522,ÃœrÃ¼mqi,China,44.20N,87.20E
1,1990,-10.704,0.399,ÃœrÃ¼mqi,China,44.20N,87.20E
2,1990,-0.075,0.469,ÃœrÃ¼mqi,China,44.20N,87.20E
3,1990,9.557,0.249,ÃœrÃ¼mqi,China,44.20N,87.20E
4,1990,17.972,0.285,ÃœrÃ¼mqi,China,44.20N,87.20E
...,...,...,...,...,...,...,...
46026,2013,8.374,0.199,Bonn,Germany,50.63N,6.34E
46027,2013,11.463,0.235,Bonn,Germany,50.63N,6.34E
46028,2013,15.589,0.247,Bonn,Germany,50.63N,6.34E
46029,2013,19.869,0.280,Bonn,Germany,50.63N,6.34E


In [30]:
grouped = country.groupby(['dt', 'Country']).mean()

In [31]:
grouped

AverageTemperature  AverageTemperatureUncertainty
dt   Country                                                          
1990 China                    10.930208                       0.331760
     Germany                   9.785250                       0.289024
     India                    26.033152                       0.395594
     Japan                    14.147424                       0.269750
     United Kingdom           10.101881                       0.279119
...                                 ...                            ...
2013 Germany                   9.141339                       0.279643
     India                    27.179701                       0.512657
     Japan                    13.804729                       0.450458
     United Kingdom            9.156000                       0.434500
     United States            14.929293                       0.387619

[144 rows x 2 columns]